# Importing required libraries

In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import json
import nltk
import re
import csv
import matplotlib.pyplot as plt 
import seaborn as sns
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import tensorflow as tf
from tensorflow import keras
import scipy
from keras.optimizers import Adam, SGD
from sklearn.metrics import confusion_matrix
import itertools
from sklearn.feature_extraction.text import CountVectorizer
import gensim
import gensim.downloader as gensim_api

# Importing saved dataset and applying word2vectors

In [2]:
data=pd.read_csv("movie.csv")
data

,Unnamed: 0,movie_id,synopsis,movie_name,genre,genre_new,filtered_plot
0,0,23890098,"Shlykov, a hard-working taxi driver and Lyosha...",Taxi Blues,"{""/m/07s9rl0"": ""Drama"", ""/m/03q4nz"": ""World ci...","['Drama', 'World cinema']",shlykov hard working taxi driver lyosha saxoph...
1,1,31186339,The nation of Panem consists of a wealthy Capi...,The Hunger Games,"{""/m/03btsm8"": ""Action/Adventure"", ""/m/06n90"":...","['Action/Adventure', 'Science Fiction', 'Actio...",nation panem consists wealthy capitol twelve p...
2,2,20663735,Poovalli Induchoodan is sentenced for six yea...,Narasimham,"{""/m/04t36"": ""Musical"", ""/m/02kdv5l"": ""Action""...","['Musical', 'Action', 'Drama', 'Bollywood']",poovalli induchoodan sentenced six year prison...
3,3,2231378,"The Lemon Drop Kid , a New York City swindler,...",The Lemon Drop Kid,"{""/m/06qm3"": ""Screwball comedy"", ""/m/01z4y"": ""...","['Screwball comedy', 'Comedy']",lemon drop kid new york city swindler illegall...
4,4,595909,Seventh-day Adventist Church pastor Michael Ch...,A Cry in the Dark,"{""/m/0lsxr"": ""Crime Fiction"", ""/m/07s9rl0"": ""D...","['Crime Fiction', 'Drama', 'Docudrama', 'World...",seventh day adventist church pastor michael ch...
...,...,...,...,...,...,...,...
41788,42199,34808485,"The story is about Reema , a young Muslim scho...",Oomakkuyil Padumbol,"{""/m/0bj8m2"": ""Children's""}","[""Children's""]",story reema young muslim schoolgirl malabar lo...
41789,42200,1096473,"In 1928 Hollywood, director Leo Andreyev look...",The Last Command,"{""/m/06ppq"": ""Silent film"", ""/m/0219x_"": ""Indi...","['Silent film', 'Indie', 'Black-and-white', 'P...",hollywood director leo andreyev look photograp...
41790,42201,35102018,American Luthier focuses on Randy Parsons’ tra...,Randy Parsons: American Luthier,"{""/m/02hmvc"": ""Short Film"", ""/m/04rlf"": ""Music...","['Short Film', 'Music', 'Biographical film', '...",american luthier focus randy parson transforma...
41791,42202,8628195,"Abdur Rehman Khan , a middle-aged dry fruit se...",Kabuliwala,"{""/m/07s9rl0"": ""Drama""}",['Drama'],abdur rehman khan middle aged dry fruit seller...


In [3]:
data1 = data.sample(frac=0.1)
data1

,Unnamed: 0,movie_id,synopsis,movie_name,genre,genre_new,filtered_plot
5205,5259,10726300,Thakur Bhupendra Singh and his son Ajit are ...,Aaj Ka Arjun,"{""/m/0lsxr"": ""Crime Fiction"", ""/m/03q4nz"": ""Wo...","['Crime Fiction', 'World cinema', 'Musical', '...",thakur bhupendra singh son ajit landlord presi...
19614,19823,31585384,Writer Lauren Cochran suffers from agoraphobi...,The Nesting,"{""/m/03npn"": ""Horror""}",['Horror'],writer lauren cochran suffers agoraphobia bid ...
40534,40935,4062150,Okuyama's face was disfigured in an industrial...,Tanin no kao,"{""/m/06n90"": ""Science Fiction"", ""/m/0gw5n2f"": ...","['Science Fiction', 'Japanese Movies', 'Surrea...",okuyamas face disfigured industrial accident f...
15345,15506,33569921,"As students Fritz , Ulrich and Johannes Wolf ...",Immer die Radfahrer,"{""/m/05p553"": ""Comedy film""}",['Comedy film'],student fritz ulrich johannes wolf albach rett...
30262,30564,18568161,"Halo revolves around a seven year old girl, Sa...",Halo,"{""/m/07s9rl0"": ""Drama""}",['Drama'],halo revolves around seven year old girl sasha...
...,...,...,...,...,...,...,...
7355,7431,24465420,British Captain Fred Allison bids farewell to...,Captured!,"{""/m/082gq"": ""War film""}",['War film'],british captain fred allison bid farewell new ...
7105,7178,9701325,The President of the United States nominates ...,Advise and Consent,"{""/m/0hn10"": ""LGBT"", ""/m/07s9rl0"": ""Drama"", ""/...","['LGBT', 'Drama', 'Political drama', 'Courtroo...",president united state nominates robert leffin...
31613,31927,4848029,Jim Doyle is a maverick mathematician who has...,The Bank,"{""/m/0lsxr"": ""Crime Fiction"", ""/m/02wtdps"": ""C...","['Crime Fiction', 'Crime Thriller', 'Drama', '...",jim doyle maverick mathematician devised formu...
15157,15312,2565462,"In student lodgings in Munich, Sophie Scholl a...",Sophie Scholl – The Final Days,"{""/m/0lsxr"": ""Crime Fiction"", ""/m/03q4nz"": ""Wo...","['Crime Fiction', 'World cinema', 'History', '...",student lodging munich sophie scholl close fri...


In [2]:
word_vecs = gensim_api.load("word2vec-google-news-300")

In [5]:
word_vecs.index_to_key

['</s>',
 'in',
 'for',
 'that',
 'is',
 'on',
 '##',
 'The',
 'with',
 'said',
 'was',
 'the',
 'at',
 'not',
 'as',
 'it',
 'be',
 'from',
 'by',
 'are',
 'I',
 'have',
 'he',
 'will',
 'has',
 '####',
 'his',
 'an',
 'this',
 'or',
 'their',
 'who',
 'they',
 'but',
 '$',
 'had',
 'year',
 'were',
 'we',
 'more',
 '###',
 'up',
 'been',
 'you',
 'its',
 'one',
 'about',
 'would',
 'which',
 'out',
 'can',
 'It',
 'all',
 'also',
 'two',
 'after',
 'first',
 'He',
 'do',
 'time',
 'than',
 'when',
 'We',
 'over',
 'last',
 'new',
 'other',
 'her',
 'people',
 'into',
 'In',
 'our',
 'there',
 'A',
 'she',
 'could',
 'just',
 'years',
 'some',
 'U.S.',
 'three',
 'million',
 'them',
 'what',
 'But',
 'so',
 'no',
 'like',
 'if',
 'only',
 'percent',
 'get',
 'did',
 'him',
 'game',
 'back',
 'because',
 'now',
 '#.#',
 'before',
 'company',
 'any',
 'team',
 'against',
 'off',
 'This',
 'most',
 'made',
 'through',
 'make',
 'second',
 'state',
 'well',
 'day',
 'season',
 'says',
 'w

In [3]:
def sum_word2vec(synopsis,word_vecs):
    words = synopsis.split(" ")
    l = []
    for i in words:
        if i in word_vecs.index_to_key:
            l.append(word_vecs[i])
    add = sum(l)
    return add

In [11]:
data1['sum_w2v'] = data1['filtered_plot'].apply(lambda x:sum_word2vec(x,word_vecs))
data1

,Unnamed: 0,movie_id,synopsis,movie_name,genre,genre_new,filtered_plot,sum_w2v
5205,5259,10726300,Thakur Bhupendra Singh and his son Ajit are ...,Aaj Ka Arjun,"{""/m/0lsxr"": ""Crime Fiction"", ""/m/03q4nz"": ""Wo...","['Crime Fiction', 'World cinema', 'Musical', '...",thakur bhupendra singh son ajit landlord presi...,"[2.556839, 3.1699066, 2.5140228, 0.453537, -3...."
19614,19823,31585384,Writer Lauren Cochran suffers from agoraphobi...,The Nesting,"{""/m/03npn"": ""Horror""}",['Horror'],writer lauren cochran suffers agoraphobia bid ...,"[1.8339844, 1.3442383, -0.36816406, 0.657959, ..."
40534,40935,4062150,Okuyama's face was disfigured in an industrial...,Tanin no kao,"{""/m/06n90"": ""Science Fiction"", ""/m/0gw5n2f"": ...","['Science Fiction', 'Japanese Movies', 'Surrea...",okuyamas face disfigured industrial accident f...,"[15.684422, 9.671326, 4.4506836, 11.867134, -1..."
15345,15506,33569921,"As students Fritz , Ulrich and Johannes Wolf ...",Immer die Radfahrer,"{""/m/05p553"": ""Comedy film""}",['Comedy film'],student fritz ulrich johannes wolf albach rett...,"[4.185997, 0.38246155, -0.7231293, 5.5049744, ..."
30262,30564,18568161,"Halo revolves around a seven year old girl, Sa...",Halo,"{""/m/07s9rl0"": ""Drama""}",['Drama'],halo revolves around seven year old girl sasha...,"[6.783325, 2.7180328, -0.76277924, 4.7471924, ..."
...,...,...,...,...,...,...,...,...
7355,7431,24465420,British Captain Fred Allison bids farewell to...,Captured!,"{""/m/082gq"": ""War film""}",['War film'],british captain fred allison bid farewell new ...,"[6.591466, 12.522953, 3.6369324, 7.449337, -5...."
7105,7178,9701325,The President of the United States nominates ...,Advise and Consent,"{""/m/0hn10"": ""LGBT"", ""/m/07s9rl0"": ""Drama"", ""/...","['LGBT', 'Drama', 'Political drama', 'Courtroo...",president united state nominates robert leffin...,"[3.0679092, 5.4014225, 7.3239803, 4.198738, -3..."
31613,31927,4848029,Jim Doyle is a maverick mathematician who has...,The Bank,"{""/m/0lsxr"": ""Crime Fiction"", ""/m/02wtdps"": ""C...","['Crime Fiction', 'Crime Thriller', 'Drama', '...",jim doyle maverick mathematician devised formu...,"[0.21972656, 0.42459106, -0.5483322, 2.6157227..."
15157,15312,2565462,"In student lodgings in Munich, Sophie Scholl a...",Sophie Scholl – The Final Days,"{""/m/0lsxr"": ""Crime Fiction"", ""/m/03q4nz"": ""Wo...","['Crime Fiction', 'World cinema', 'History', '...",student lodging munich sophie scholl close fri...,"[16.906288, 16.096968, 24.503977, 21.56551, -1..."


In [4]:
def avg_word2vec(synopsis,word_vecs):
    words = synopsis.split(" ")
    l = []
    for i in words:
        if i in word_vecs.index_to_key:
            l.append(word_vecs[i])
    avg = sum(l)/len(l)
    return avg

In [13]:
data1['avg_w2v'] = data1['filtered_plot'].apply(lambda x:avg_word2vec(x,word_vecs))
data1

,Unnamed: 0,movie_id,synopsis,movie_name,genre,genre_new,filtered_plot,sum_w2v,avg_w2v
5205,5259,10726300,Thakur Bhupendra Singh and his son Ajit are ...,Aaj Ka Arjun,"{""/m/0lsxr"": ""Crime Fiction"", ""/m/03q4nz"": ""Wo...","['Crime Fiction', 'World cinema', 'Musical', '...",thakur bhupendra singh son ajit landlord presi...,"[2.556839, 3.1699066, 2.5140228, 0.453537, -3....","[0.035511654, 0.04402648, 0.034916982, 0.00629..."
19614,19823,31585384,Writer Lauren Cochran suffers from agoraphobi...,The Nesting,"{""/m/03npn"": ""Horror""}",['Horror'],writer lauren cochran suffers agoraphobia bid ...,"[1.8339844, 1.3442383, -0.36816406, 0.657959, ...","[0.07335938, 0.053769533, -0.014726562, 0.0263..."
40534,40935,4062150,Okuyama's face was disfigured in an industrial...,Tanin no kao,"{""/m/06n90"": ""Science Fiction"", ""/m/0gw5n2f"": ...","['Science Fiction', 'Japanese Movies', 'Surrea...",okuyamas face disfigured industrial accident f...,"[15.684422, 9.671326, 4.4506836, 11.867134, -1...","[0.07842211, 0.04835663, 0.022253418, 0.059335..."
15345,15506,33569921,"As students Fritz , Ulrich and Johannes Wolf ...",Immer die Radfahrer,"{""/m/05p553"": ""Comedy film""}",['Comedy film'],student fritz ulrich johannes wolf albach rett...,"[4.185997, 0.38246155, -0.7231293, 5.5049744, ...","[0.05507891, 0.005032389, -0.009514859, 0.0724..."
30262,30564,18568161,"Halo revolves around a seven year old girl, Sa...",Halo,"{""/m/07s9rl0"": ""Drama""}",['Drama'],halo revolves around seven year old girl sasha...,"[6.783325, 2.7180328, -0.76277924, 4.7471924, ...","[0.079803824, 0.031976856, -0.008973873, 0.055..."
...,...,...,...,...,...,...,...,...,...
7355,7431,24465420,British Captain Fred Allison bids farewell to...,Captured!,"{""/m/082gq"": ""War film""}",['War film'],british captain fred allison bid farewell new ...,"[6.591466, 12.522953, 3.6369324, 7.449337, -5....","[0.030098017, 0.057182435, 0.016606998, 0.0340..."
7105,7178,9701325,The President of the United States nominates ...,Advise and Consent,"{""/m/0hn10"": ""LGBT"", ""/m/07s9rl0"": ""Drama"", ""/...","['LGBT', 'Drama', 'Political drama', 'Courtroo...",president united state nominates robert leffin...,"[3.0679092, 5.4014225, 7.3239803, 4.198738, -3...","[0.007866434, 0.013849801, 0.018779436, 0.0107..."
31613,31927,4848029,Jim Doyle is a maverick mathematician who has...,The Bank,"{""/m/0lsxr"": ""Crime Fiction"", ""/m/02wtdps"": ""C...","['Crime Fiction', 'Crime Thriller', 'Drama', '...",jim doyle maverick mathematician devised formu...,"[0.21972656, 0.42459106, -0.5483322, 2.6157227...","[0.011564556, 0.022346899, -0.028859591, 0.137..."
15157,15312,2565462,"In student lodgings in Munich, Sophie Scholl a...",Sophie Scholl – The Final Days,"{""/m/0lsxr"": ""Crime Fiction"", ""/m/03q4nz"": ""Wo...","['Crime Fiction', 'World cinema', 'History', '...",student lodging munich sophie scholl close fri...,"[16.906288, 16.096968, 24.503977, 21.56551, -1...","[0.029817088, 0.028389715, 0.04321689, 0.03803..."


In [15]:
#data1.to_csv("word2vec_movies.csv")

In [5]:
data1 = pd.read_csv("word2vec_movies.csv")
data1

,Unnamed: 0,Unnamed: 0.1,movie_id,synopsis,movie_name,genre,genre_new,filtered_plot,sum_w2v,avg_w2v
0,5205,5259,10726300,Thakur Bhupendra Singh and his son Ajit are ...,Aaj Ka Arjun,"{""/m/0lsxr"": ""Crime Fiction"", ""/m/03q4nz"": ""Wo...","['Crime Fiction', 'World cinema', 'Musical', '...",thakur bhupendra singh son ajit landlord presi...,[ 2.556839 3.1699066 2.5140228 0.45...,[ 0.03551165 0.04402648 0.03491698 0.006299...
1,19614,19823,31585384,Writer Lauren Cochran suffers from agoraphobi...,The Nesting,"{""/m/03npn"": ""Horror""}",['Horror'],writer lauren cochran suffers agoraphobia bid ...,[ 1.83398438e+00 1.34423828e+00 -3.68164062e-...,[ 7.33593777e-02 5.37695326e-02 -1.47265624e-...
2,40534,40935,4062150,Okuyama's face was disfigured in an industrial...,Tanin no kao,"{""/m/06n90"": ""Science Fiction"", ""/m/0gw5n2f"": ...","['Science Fiction', 'Japanese Movies', 'Surrea...",okuyamas face disfigured industrial accident f...,[ 15.684422 9.671326 4.4506836 11.86...,[ 0.07842211 0.04835663 0.02225342 0.059335...
3,15345,15506,33569921,"As students Fritz , Ulrich and Johannes Wolf ...",Immer die Radfahrer,"{""/m/05p553"": ""Comedy film""}",['Comedy film'],student fritz ulrich johannes wolf albach rett...,[ 4.185997 0.38246155 -0.7231293 5.50...,[ 0.05507891 0.00503239 -0.00951486 0.072433...
4,30262,30564,18568161,"Halo revolves around a seven year old girl, Sa...",Halo,"{""/m/07s9rl0"": ""Drama""}",['Drama'],halo revolves around seven year old girl sasha...,[ 6.78332520e+00 2.71803284e+00 -7.62779236e-...,[ 7.98038244e-02 3.19768563e-02 -8.97387322e-...
...,...,...,...,...,...,...,...,...,...,...
4174,7355,7431,24465420,British Captain Fred Allison bids farewell to...,Captured!,"{""/m/082gq"": ""War film""}",['War film'],british captain fred allison bid farewell new ...,[ 6.591466 12.522953 3.6369324 7.44...,[ 0.03009802 0.05718243 0.016607 0.034015...
4175,7105,7178,9701325,The President of the United States nominates ...,Advise and Consent,"{""/m/0hn10"": ""LGBT"", ""/m/07s9rl0"": ""Drama"", ""/...","['LGBT', 'Drama', 'Political drama', 'Courtroo...",president united state nominates robert leffin...,[ 3.06790924e+00 5.40142250e+00 7.32398033e+...,[ 7.86643382e-03 1.38498014e-02 1.87794361e-...
4176,31613,31927,4848029,Jim Doyle is a maverick mathematician who has...,The Bank,"{""/m/0lsxr"": ""Crime Fiction"", ""/m/02wtdps"": ""C...","['Crime Fiction', 'Crime Thriller', 'Drama', '...",jim doyle maverick mathematician devised formu...,[ 2.1972656e-01 4.2459106e-01 -5.4833221e-01 ...,[ 1.15645556e-02 2.23468989e-02 -2.88595911e-...
4177,15157,15312,2565462,"In student lodgings in Munich, Sophie Scholl a...",Sophie Scholl – The Final Days,"{""/m/0lsxr"": ""Crime Fiction"", ""/m/03q4nz"": ""Wo...","['Crime Fiction', 'World cinema', 'History', '...",student lodging munich sophie scholl close fri...,[ 16.906288 16.096968 24.503977 21.56...,[ 0.02981709 0.02838971 0.04321689 0.038034...


In [6]:
genres = [] 

for i in data1['genre']: 
  genres.append(list(json.loads(i).values()))

data1['genre_list'] = genres

genres_list = sum(genres,[])
print("No of Genres: ",len(set(genres_list)))

No of Genres:  282


In [7]:
multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(data1['genre_list'])
y = multilabel_binarizer.transform(data1['genre_list'])
y

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [8]:
multilabel_binarizer.classes_.shape

(282,)

In [9]:
def adjuster(x):
    l=[]
    x=x.replace('\n', '')[3:-1].split(' ')
    for i in x:
        if i!='':
            l.append(float(i))
    return np.array(l)

data1['avg_w2v_final'] = data1['avg_w2v'].apply(lambda x:adjuster(x))
data1['sum_w2v_final'] = data1['sum_w2v'].apply(lambda x:adjuster(x))
data1

,Unnamed: 0,Unnamed: 0.1,movie_id,synopsis,movie_name,genre,genre_new,filtered_plot,sum_w2v,avg_w2v,genre_list,avg_w2v_final,sum_w2v_final
0,5205,5259,10726300,Thakur Bhupendra Singh and his son Ajit are ...,Aaj Ka Arjun,"{""/m/0lsxr"": ""Crime Fiction"", ""/m/03q4nz"": ""Wo...","['Crime Fiction', 'World cinema', 'Musical', '...",thakur bhupendra singh son ajit landlord presi...,[ 2.556839 3.1699066 2.5140228 0.45...,[ 0.03551165 0.04402648 0.03491698 0.006299...,"[Crime Fiction, World cinema, Musical, Drama, ...","[0.03551165, 0.04402648, 0.03491698, 0.0062991...","[2.556839, 3.1699066, 2.5140228, 0.453537, -3...."
1,19614,19823,31585384,Writer Lauren Cochran suffers from agoraphobi...,The Nesting,"{""/m/03npn"": ""Horror""}",['Horror'],writer lauren cochran suffers agoraphobia bid ...,[ 1.83398438e+00 1.34423828e+00 -3.68164062e-...,[ 7.33593777e-02 5.37695326e-02 -1.47265624e-...,[Horror],"[0.0033593777, 0.0537695326, -0.0147265624, 0....","[0.83398438, 1.34423828, -0.368164062, 0.65795..."
2,40534,40935,4062150,Okuyama's face was disfigured in an industrial...,Tanin no kao,"{""/m/06n90"": ""Science Fiction"", ""/m/0gw5n2f"": ...","['Science Fiction', 'Japanese Movies', 'Surrea...",okuyamas face disfigured industrial accident f...,[ 15.684422 9.671326 4.4506836 11.86...,[ 0.07842211 0.04835663 0.02225342 0.059335...,"[Science Fiction, Japanese Movies, Surrealism,...","[0.07842211, 0.04835663, 0.02225342, 0.0593356...","[5.684422, 9.671326, 4.4506836, 11.867134, -15..."
3,15345,15506,33569921,"As students Fritz , Ulrich and Johannes Wolf ...",Immer die Radfahrer,"{""/m/05p553"": ""Comedy film""}",['Comedy film'],student fritz ulrich johannes wolf albach rett...,[ 4.185997 0.38246155 -0.7231293 5.50...,[ 0.05507891 0.00503239 -0.00951486 0.072433...,[Comedy film],"[0.05507891, 0.00503239, -0.00951486, 0.072433...","[4.185997, 0.38246155, -0.7231293, 5.5049744, ..."
4,30262,30564,18568161,"Halo revolves around a seven year old girl, Sa...",Halo,"{""/m/07s9rl0"": ""Drama""}",['Drama'],halo revolves around seven year old girl sasha...,[ 6.78332520e+00 2.71803284e+00 -7.62779236e-...,[ 7.98038244e-02 3.19768563e-02 -8.97387322e-...,[Drama],"[0.0098038244, 0.0319768563, -0.00897387322, 0...","[0.7833252, 2.71803284, -0.762779236, 4.747192..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4174,7355,7431,24465420,British Captain Fred Allison bids farewell to...,Captured!,"{""/m/082gq"": ""War film""}",['War film'],british captain fred allison bid farewell new ...,[ 6.591466 12.522953 3.6369324 7.44...,[ 0.03009802 0.05718243 0.016607 0.034015...,[War film],"[0.03009802, 0.05718243, 0.016607, 0.03401524,...","[6.591466, 12.522953, 3.6369324, 7.449337, -5...."
4175,7105,7178,9701325,The President of the United States nominates ...,Advise and Consent,"{""/m/0hn10"": ""LGBT"", ""/m/07s9rl0"": ""Drama"", ""/...","['LGBT', 'Drama', 'Political drama', 'Courtroo...",president united state nominates robert leffin...,[ 3.06790924e+00 5.40142250e+00 7.32398033e+...,[ 7.86643382e-03 1.38498014e-02 1.87794361e-...,"[LGBT, Drama, Political drama, Courtroom Drama]","[0.00086643382, 0.0138498014, 0.0187794361, 0....","[0.06790924, 5.4014225, 7.32398033, 4.1987381,..."
4176,31613,31927,4848029,Jim Doyle is a maverick mathematician who has...,The Bank,"{""/m/0lsxr"": ""Crime Fiction"", ""/m/02wtdps"": ""C...","['Crime Fiction', 'Crime Thriller', 'Drama', '...",jim doyle maverick mathematician devised formu...,[ 2.1972656e-01 4.2459106e-01 -5.4833221e-01 ...,[ 1.15645556e-02 2.23468989e-02 -2.88595911e-...,"[Crime Fiction, Crime Thriller, Drama, Suspens...","[0.0015645556, 0.0223468989, -0.0288595911, 0....","[0.01972656, 0.42459106, -0.54833221, 2.615722..."
4177,15157,15312,2565462,"In student lodgings in Munich, Sophie Scholl a...",Sophie Scholl – The Final Days,"{""/m/0lsxr"": ""Crime Fiction"", ""/m/03q4nz"": ""Wo...","['Crime Fiction', 'World cinema', 'History', '...",student lodging munich sophie scholl close fri...,[ 16.906288 16.096

# Importing ML libraries

In [10]:
from sklearn.linear_model import LogisticRegression

from sklearn.multiclass import OneVsRestClassifier

from sklearn.model_selection import cross_validate

from sklearn.metrics import f1_score,log_loss,precision_score,recall_score,make_scorer,accuracy_score

# Initiating the Model for Sum of Word2vectors and using cross-validation

In [76]:
lr = LogisticRegression()
clf = OneVsRestClassifier(lr)

In [77]:
scoring = {'accuracy': make_scorer(accuracy_score),
           'precision': make_scorer(precision_score, average = 'micro'),
           'recall': make_scorer(recall_score, average = 'micro'),
           'f1': make_scorer(f1_score, average = 'micro'),
           'log_loss': make_scorer(log_loss)}

In [78]:
scores = cross_validate(clf, np.array(data1['sum_w2v_final'].tolist()), y, cv=5,scoring=scoring,return_train_score=True)
scores

{'fit_time': array([31.70759082, 37.267946  , 36.61139917, 32.73178267, 38.41787434]),
 'score_time': array([0.307302  , 0.28510618, 0.34414506, 0.30348778, 0.24035764]),
 'test_accuracy': array([0.01794258, 0.02272727, 0.01913876, 0.02392344, 0.02035928]),
 'train_accuracy': array([0.21447801, 0.20580317, 0.20849536, 0.21717021, 0.22069378]),
 'test_precision': array([0.31738318, 0.31419624, 0.33883677, 0.33435348, 0.30673953]),
 'train_precision': array([0.84059033, 0.82981773, 0.83744206, 0.84503142, 0.84319191]),
 'test_recall': array([0.27547047, 0.31202488, 0.29044709, 0.28184457, 0.2781632 ]),
 'train_recall': array([0.66202292, 0.6578456 , 0.65672505, 0.65505284, 0.67027737]),
 'test_f1': array([0.29494528, 0.3131068 , 0.31278143, 0.30586177, 0.29175329]),
 'train_f1': array([0.74069633, 0.73389178, 0.73615484, 0.73801219, 0.74685686]),
 'test_log_loss': array([88.20630704, 79.32167255, 86.85597928, 87.89164413, 87.02353193]),
 'train_log_loss': array([41.08433903, 42.90753365,

In [79]:
print("The Fit Time is: ",scores['fit_time'].mean())
print("\n")
print("The Score Time is: ",scores['score_time'].mean())
print("\n")
print("The Train Accuracy score is: ",scores['train_accuracy'].mean())
print("\n")
print("The Train F1 score is: ",scores['train_f1'].mean())
print("\n")
print("The Train Precision score is: ",scores['train_precision'].mean())
print("\n")
print("The Train Recall score is: ",scores['train_recall'].mean())
print("\n")
print("The Train Log Loss is: ",scores['train_log_loss'].mean())
print("\n")
print("The Test Accuracy score is: ",scores['test_accuracy'].mean())
print("\n")
print("The Test F1 score is: ",scores['test_f1'].mean())
print("\n")
print("The Test Precision score is: ",scores['test_precision'].mean())
print("\n")
print("The Test Recall score is: ",scores['test_recall'].mean())
print("\n")
print("The Test Log Loss is: ",scores['test_log_loss'].mean())
print("\n")

The Fit Time is:  35.34731860160828


The Score Time is:  0.2960797309875488


The Train Accuracy score is:  0.21332810686330217


The Train F1 score is:  0.7391224000153741


The Train Precision score is:  0.8392146893102602


The Train Recall score is:  0.6603847556091423


The Train Log Loss is:  41.737645335145146


The Test Accuracy score is:  0.020818267770678738


The Test F1 score is:  0.3036897146331038


The Test Precision score is:  0.32230184007816753


The Test Recall score is:  0.28759004120586046


The Test Log Loss is:  85.85982698536152




# Initiating the Model for Average of Word2vectors and using cross-validation

In [80]:
lr = LogisticRegression()
clf = OneVsRestClassifier(lr)

In [81]:
scoring = {'accuracy': make_scorer(accuracy_score),
           'precision': make_scorer(precision_score, average = 'micro'),
           'recall': make_scorer(recall_score, average = 'micro'),
           'f1': make_scorer(f1_score, average = 'micro'),
           'log_loss': make_scorer(log_loss)}

In [82]:
scores = cross_validate(clf, np.array(data1['avg_w2v_final'].tolist()), y, cv=5,scoring=scoring,return_train_score=True)
scores

{'fit_time': array([10.71187592,  9.09144664,  8.38977051,  7.49994063,  8.08599591]),
 'score_time': array([0.24434328, 0.29812646, 0.21999955, 0.23185706, 0.22876906]),
 'test_accuracy': array([0.04665072, 0.05143541, 0.04904306, 0.0562201 , 0.0491018 ]),
 'train_accuracy': array([0.05833084, 0.05623691, 0.05623691, 0.05354472, 0.05352871]),
 'test_precision': array([0.61612284, 0.62842466, 0.64227642, 0.65454545, 0.6440367 ]),
 'train_precision': array([0.69181233, 0.68687826, 0.68561873, 0.68783586, 0.68211921]),
 'test_recall': array([0.10415315, 0.1268141 , 0.1016404 , 0.10448243, 0.11595639]),
 'train_recall': array([0.12397989, 0.11770436, 0.11855585, 0.11624835, 0.11833251]),
 'test_f1': array([0.17818485, 0.21104083, 0.1755068 , 0.18020022, 0.19652856]),
 'train_f1': array([0.21027613, 0.2009702 , 0.20215538, 0.1988841 , 0.20167832]),
 'test_log_loss': array([61.67629313, 57.28490718, 59.96483697, 61.0105623 , 59.73653603]),
 'train_log_loss': array([61.10632093, 60.79686679,

In [83]:
print("The Fit Time is: ",scores['fit_time'].mean())
print("\n")
print("The Score Time is: ",scores['score_time'].mean())
print("\n")
print("The Train Accuracy score is: ",scores['train_accuracy'].mean())
print("\n")
print("The Train F1 score is: ",scores['train_f1'].mean())
print("\n")
print("The Train Precision score is: ",scores['train_precision'].mean())
print("\n")
print("The Train Recall score is: ",scores['train_recall'].mean())
print("\n")
print("The Train Log Loss is: ",scores['train_log_loss'].mean())
print("\n")
print("The Test Accuracy score is: ",scores['test_accuracy'].mean())
print("\n")
print("The Test F1 score is: ",scores['test_f1'].mean())
print("\n")
print("The Test Precision score is: ",scores['test_precision'].mean())
print("\n")
print("The Test Recall score is: ",scores['test_recall'].mean())
print("\n")
print("The Test Log Loss is: ",scores['test_log_loss'].mean())
print("\n")

The Fit Time is:  8.755805921554565


The Score Time is:  0.24461908340454103


The Train Accuracy score is:  0.05557561898246283


The Train F1 score is:  0.2027928273603697


The Train Precision score is:  0.6868528752008962


The Train Recall score is:  0.11896419122649862


The Train Log Loss is:  59.706683741814594


The Test Accuracy score is:  0.05049021574076727


The Test F1 score is:  0.18829225111799863


The Test Precision score is:  0.6370812145565228


The Test Recall score is:  0.11060929235501223


The Test Log Loss is:  59.93462712257597




# Implementing a testing module for all four models

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import CountVectorizer

In [12]:
tfidf_vectorizer = TfidfVectorizer()
x_tfidf = tfidf_vectorizer.fit_transform(data1['filtered_plot'])
print(x_tfidf.shape,y.shape)

(4179, 41619) (4179, 282)


In [13]:
bow_vectorizer = CountVectorizer()
x_bow = bow_vectorizer.fit_transform(data1['filtered_plot'])
print(x_bow.shape,y.shape)

(4179, 41619) (4179, 282)


In [14]:
lr_tfidf = LogisticRegression()
clf_tfidf = OneVsRestClassifier(lr_tfidf)
clf_tfidf.fit(x_tfidf, y)

OneVsRestClassifier(estimator=LogisticRegression())

In [15]:
lr_bow = LogisticRegression()
clf_bow = OneVsRestClassifier(lr_bow)
clf_bow.fit(x_bow, y)

OneVsRestClassifier(estimator=LogisticRegression())

In [16]:
lr_w2v_sum = LogisticRegression()
clf_w2v_sum = OneVsRestClassifier(lr_w2v_sum)
clf_w2v_sum.fit(data1['sum_w2v_final'].tolist(), y)

OneVsRestClassifier(estimator=LogisticRegression())

In [17]:
lr_w2v_avg = LogisticRegression()
clf_w2v_avg = OneVsRestClassifier(lr_w2v_avg)
clf_w2v_avg.fit(data1['avg_w2v_final'].tolist(), y)

OneVsRestClassifier(estimator=LogisticRegression())

In [18]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\PREDATOR08\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\PREDATOR08\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [19]:
eng_stopwords = set(stopwords.words('english'))

def filter_synopsis(synopsis):
    synopsis = re.sub("\'", "", synopsis) 
    synopsis = re.sub("[^a-zA-Z]"," ",synopsis) 
    synopsis = ' '.join(synopsis.split())
    synopsis = synopsis.lower() 
    
    return synopsis

def stopwords_filtering(plot):
    filtered_text = [x for x in plot.split() if not x in eng_stopwords]
    return ' '.join(filtered_text)

def lemmatization_plot(plot):
    lemmatizer = WordNetLemmatizer()
    filtered_text = [lemmatizer.lemmatize(x) for x in plot.split()]
    return ' '.join(filtered_text)

In [20]:
text = "The film commences with a portrait of society's caste divide, with the police checking the caste of detainees to determine who should be released. Only inmates from the upper and middle castes get released. The police blame people from lower caste accusing them of perpetrating crimes in multiple pending cases.[6] Rajakannu and Sengeni are a couple from the Irula tribe who labour in the fields of upper-caste men to control rat infestation and catch venomous snakes. Rajakannu is called to a rich man's dwelling to catch a snake that sneaked inside a room. The next day, a case of burglary is reported when the wife of the man reports missing jewellery pieces from her closet and raises doubt on Rajakannu. The police invade Rajakannu's home to find evidence. Rajakannu had left the town earlier for work. During the invasion, the police brutally beat and unlawfully detains his pregnant wife. The police arrest other relatives, Rajakannu's brother, Iruttupan, his sister Pachaiammal and his brother-in-law, Mosukutty and torture them to confess the whereabouts of Rajakannu. The police find Rajakannu and imprison him. They brutally torture him to confess the crime but releases his wife. Later, Sengeni is informed that all three men in detention are absconding and police threaten her furthermore to tell them about their whereabouts.Mythra, who teaches adults from Irula tribe, gets to know about Chandru, a lawyer who fights cases for tribal communities, and manages to convince him to seek justice for Sengeni. After listening to the narration of all events that happened until then from Sengeni, Chandru files a habeas corpus case in the court. The public attorney, in defence of the police, produces eye-witnesses to concoct a story that Rajakannu and the other two indeed absconded from jail. Finding gaps in the eye-witnesses' statements, Chandru uncovers that they were committing perjury and asks the court to investigate two police officers – Sub-Inspector Gurumurthy, a casteist cop and constable Veerasamy.Advocate general S. Ram Mohan takes over the case and in defence of the police, claims that the three accused have fled to Kerala when Varadarajulu, the employer of Iruttapan, acknowledges that Iruttapan informed him through a phone call that he fled to Kerala after committing robbery. In retort, Chandru manages to discover that the three cops in question went to Kerala to make a phone call to Varadarajulu which Gurumurthy admits, that he mimicked the voice of Iruttapan. Chandru and Mythra find Rajakannu dead in the middle of a road near the border of Pondicherry. Both believe that Rajakannu did not die due to a car accident, rather due to lock-up murder. Chandru consults the doctor who operated on Rajakannu. The doctor tells him that the cause of death was due to the breaking of the ribcage, but theorises that this could be caused by a car running over him. Veerasamy confesses to Ram Mohan that Rajakannu died in lock up custody. Veerasamy called Guru after the death. Guru tells Veerasamy that he should stage that both escaped and leave Rajakannu on the road making everyone believe that he died due to a car accident. Mosukutty and Iruttapan were transferred to another jail in Kerala. After hearing this, the lawyer tells them to act like they do not know anything in court. Chandru checks the call history of the police station and tells the court that a call to Guru's residence was made at 9:10 pm, which does not match with Veerasamy's statement. Chandru asks the court for more time to investigate, which they allow.Chandru, Mythra, Sengeni and the Irular tribe go around the city protesting for justice. Chandru connects the dots and finds that Irutappan indeed called Varadarajulu, but the police coerced him to make the call. Mythra finds Iruttapan and Mosukutty and they tell the court about the torture all three of them went through and how the cops killed Rajakannu. Perumalsamy, an Inspector general, says the cops took a bribe from the real thief. Chandru also says that on the road where Rajakannu's corpse was found, there were a set of tyre marks. There were also footprints that matched Guru's and Constable Kiruba's footprints. After hearing these shreds of evidence, the court announces the verdict: the cops who killed Rajakannu should be arrested, Sengeni will get three lakhs and half a ground land as compensation while Iruttapan, Mosukutty and Pachaiamma get two lakhs each. Sengeni tearfully thanks Chandru for his help and Chandru attends the inauguration of Sengeni's new house, fulfilling Rajakannu's dream to get Sengeni a new house."
text = filter_synopsis(text)
text = stopwords_filtering(text)
text = lemmatization_plot(text)
text

'film commences portrait society caste divide police checking caste detainee determine released inmate upper middle caste get released police blame people lower caste accusing perpetrating crime multiple pending case rajakannu sengeni couple irula tribe labour field upper caste men control rat infestation catch venomous snake rajakannu called rich man dwelling catch snake sneaked inside room next day case burglary reported wife man report missing jewellery piece closet raise doubt rajakannu police invade rajakannus home find evidence rajakannu left town earlier work invasion police brutally beat unlawfully detains pregnant wife police arrest relative rajakannus brother iruttupan sister pachaiammal brother law mosukutty torture confess whereabouts rajakannu police find rajakannu imprison brutally torture confess crime release wife later sengeni informed three men detention absconding police threaten furthermore tell whereabouts mythra teach adult irula tribe get know chandru lawyer figh

In [21]:
tfidf_text = tfidf_vectorizer.transform([text])
bow_text = bow_vectorizer.transform([text])
w2v_sum_text = np.array([sum_word2vec(text,word_vecs)])
w2v_avg_text = np.array([avg_word2vec(text,word_vecs)])

In [22]:
print("TF-IDF Model Output: ",multilabel_binarizer.inverse_transform(clf_tfidf.predict(tfidf_text)))
print("\n")
print("Bag Of Words Model Output: ",multilabel_binarizer.inverse_transform(clf_bow.predict(bow_text)))
print("\n")
print("Word2vector Sum of vectors Model Output: ",multilabel_binarizer.inverse_transform(clf_w2v_sum.predict(w2v_sum_text)))
print("\n")
print("Word2vector Average of vectors Model Output: ",multilabel_binarizer.inverse_transform(clf_w2v_sum.predict(w2v_avg_text)))

TF-IDF Model Output:  [('Drama',)]


Bag Of Words Model Output:  [('Crime Fiction', 'Crime Thriller', 'Drama', 'Mystery', 'Thriller')]


Word2vector Sum of vectors Model Output:  [('Crime Fiction',)]


Word2vector Average of vectors Model Output:  [()]
